In [32]:
from BioLink.biolink_client import BioLinkWrapper
import pandas as pd
import cache_magic

# Workflow II Rare Disease Candidates

In [33]:
from Modules.Mod0_lookups import LookUp

# workflow input is a disease identifier
lu = LookUp()

Mod O DiseaseGeneLookup metadata:
{'data_type': 'disease',
 'input_type': {'complexity': 'single', 'id_type': ['MONDO', 'DO', 'OMIM']},
 'limit': None,
 'output_type': {'complexity': 'set', 'id_type': 'HGNC'},
 'predicate': 'blm:gene associated with condition',
 'source': 'Monarch Biolink',
 'taxon': 'human'}


In [34]:
input_disease = 'MONDO:0019391' # Fanconi anemia  
input_object = {
    'input': input_disease,
    'parameters': {
        'taxon': 'human',
        'threshold': None,
    },
}

lu.load_input_object(input_object=input_object)
input_object = lu.input_object

{'description': 'Fanconi anemia (FA) is a hereditary DNA repair disorder '
                'characterized by progressive pancytopenia with bone marrow '
                'failure, variable congenital malformations and predisposition '
                'to develop hematological or solid tumors.',
 'id': 'MONDO:0019391',
 'label': 'Fanconi anemia'}


In [35]:
# get genes associated with disease from Biolink
disease_associated_genes = lu.disease_geneset_lookup()
# create list of gene curies for downstream module input
%cache input_curie_set = disease_associated_genes[['hit_id', 'hit_symbol']].to_dict(orient='records')
# # show the 
disease_associated_genes['modules'] = 'Mod0'
disease_associated_genes
# genes to investigate TNF, FANCD2OS

Loading cached value for variable 'input_curie_set'. Time since caching: 2:09:40.374452


,input_id,input_symbol,hit_id,hit_symbol,sources,modules
0,MONDO:0019391,Fanconi anemia,HGNC:1100,BRCA1,clinvar,Mod0
1,MONDO:0019391,Fanconi anemia,HGNC:1101,BRCA2,"orphane, omim, ctd, orphane, clinvar",Mod0
2,MONDO:0019391,Fanconi anemia,HGNC:11892,TNF,ctd,Mod0
3,MONDO:0019391,Fanconi anemia,HGNC:12829,XRCC2,"orphane, ctd, omim",Mod0
4,MONDO:0019391,Fanconi anemia,HGNC:20473,BRIP1,"ctd, orphane, omim, ctd, clinvar",Mod0
5,MONDO:0019391,Fanconi anemia,HGNC:20748,FANCL,"ctd, orphane, ctd, omim, clinvar",Mod0
6,MONDO:0019391,Fanconi anemia,HGNC:23168,FANCM,"ctd, orphane",Mod0
7,MONDO:0019391,Fanconi anemia,HGNC:23845,SLX4,"ctd, orphane, ctd, omim, clinvar",Mod0
8,MONDO:0019391,Fanconi anemia,HGNC:25009,UBE2T,"omim, ctd, clinvar, orphane",Mod0
9,MONDO:0019391,Fanconi anemia,HGNC:25539,RFWD3,orphane,Mod0


# Mod1A Functional Similarity
## Find similar genes based on GO functional annotations using OntoBio Jaccard similarity

In [36]:
from Modules.Mod1A_functional_sim import FunctionalSimilarity

## Mod1A_human

In [37]:
# Module specification
mod1a_input_object_human = {
    'input': input_curie_set,
    'parameters': {
        'taxon': 'human',
        'threshold': 0.75,  # jaccard index threshold
    },
}
    
func_sim_human = FunctionalSimilarity()

Mod1A Functional Similarity metadata:
{'input_type': {'complexity': 'set', 'data_type': 'gene', 'id_type': 'HGNC'},
 'output_type': {'complexity': 'set', 'data_type': 'gene', 'id_type': 'HGNC'},
 'predicate': ['blm:macromolecular machine to biological process association',
               'macromolecular machine to molecular activity association'],
 'source': 'Monarch Biolink'}


In [38]:
func_sim_human.load_input_object(mod1a_input_object_human)
func_sim_human.load_gene_set() 

In [39]:
func_sim_human.load_associations()

In [40]:
mod1a_results = func_sim_human.compute_similarity()

In [41]:
Mod1A_results_human = pd.DataFrame(mod1a_results)
Mod1A_results_human = Mod1A_results_human[~Mod1A_results_human['hit_id'].isin(disease_associated_genes['hit_id'].tolist())].sort_values('score', ascending=False)
Mod1A_results_human['module'] = 'Mod1A'
Mod1A_results_human

,hit_id,hit_symbol,input_id,input_symbol,score,module
17,HGNC:21700,RAD9B,HGNC:3583,FANCB,0.920000,Mod1A
15,HGNC:26171,FAAP100,HGNC:3583,FANCB,0.857143,Mod1A
22,HGNC:24994,INIP,HGNC:3583,FANCB,0.851852,Mod1A
26,HGNC:21700,RAD9B,HGNC:3586,FANCE,0.840000,Mod1A
20,HGNC:28467,FAAP24,HGNC:3583,FANCB,0.827586,Mod1A
16,HGNC:25021,ASTE1,HGNC:3583,FANCB,0.793103,Mod1A
18,HGNC:22223,EEPD1,HGNC:3583,FANCB,0.785714,Mod1A
25,HGNC:26171,FAAP100,HGNC:3586,FANCE,0.785714,Mod1A
8,HGNC:20922,SLX1A,HGNC:23845,SLX4,0.782313,Mod1A
27,HGNC:22223,EEPD1,HGNC:3586,FANCE,0.777778,Mod1A


# MOD1B Phenotype Similarity
## Find similar genes based on OwlSim calculated Phenotype Similarity

## Mod1B Human

In [42]:
from Modules.Mod1B1_phenotype_similarity import PhenotypeSimilarity

In [43]:
# Module specification
mod1b_input_object_human = {
    'input': input_curie_set,
     'parameters': {
        'taxon': 'human',
        'threshold': .50,
    },
}


pheno_sim_human = PhenotypeSimilarity()

Mod1B Phenotype Similarity metadata:
{'input_type': {'complexity': 'set', 'data_type': 'gene', 'id_type': 'HGNC'},
 'output_type': {'complexity': 'set', 'data_type': 'gene', 'id_type': 'HGNC'},
 'predicate': ['blm:has phenotype'],
 'source': 'Monarch Biolink'}


In [44]:
pheno_sim_human.load_input_object(mod1b_input_object_human)
pheno_sim_human.load_gene_set()

In [45]:
pheno_sim_human.load_associations()

In [46]:
mod1b_results = pheno_sim_human.compute_similarity()

In [47]:
# compute phenotype similarity
Mod1B_results = pd.DataFrame(mod1b_results)
Mod1B_results = Mod1B_results[~Mod1B_results['hit_id'].isin(disease_associated_genes['hit_id'].tolist())].sort_values('score', ascending=False)
Mod1B_results['module'] = 'Mod1B'
Mod1B_results

,hit_id,hit_symbol,input_id,input_symbol,score,module
23,HGNC:6709,LTA,HGNC:11892,TNF,0.967320,Mod1B
58,HGNC:8143,OPCML,HGNC:20473,BRIP1,0.506042,Mod1B
60,HGNC:17271,RRAS2,HGNC:20473,BRIP1,0.506042,Mod1B
65,HGNC:7499,MT-TT,HGNC:20473,BRIP1,0.506042,Mod1B
201,HGNC:8143,OPCML,HGNC:26144,PALB2,0.500747,Mod1B
203,HGNC:17271,RRAS2,HGNC:26144,PALB2,0.500747,Mod1B
208,HGNC:7499,MT-TT,HGNC:26144,PALB2,0.500747,Mod1B


# Mod1E Protein Interaction

## Mod1E Human

In [48]:
from Modules.Mod1E_interactions import GeneInteractions

In [49]:
# Module specification
mod1E_input_object_human = {
    'input': input_curie_set,
     'parameters': {
        'taxon': 'human',
        'threshold': None,
    },
}


interactions_human = GeneInteractions()

Mod1E Interaction Network metadata:
{'input_type': {'complexity': 'set', 'data_type': 'gene', 'id_type': 'HGNC'},
 'output_type': {'complexity': 'set', 'data_type': 'gene', 'id_type': 'HGNC'},
 'predicate': ['blm:interacts with'],
 'source': 'Monarch Biolink'}


In [50]:
interactions_human.load_input_object(mod1E_input_object_human)
interactions_human.load_gene_set()

In [51]:
mod1e_results = interactions_human.get_interactions()

In [52]:
Mod1E_results_human = pd.DataFrame(mod1e_results)

In [53]:
counts = Mod1E_results_human['hit_symbol'].value_counts().rename_axis('unique_values').to_frame('counts').reset_index()
high_counts = counts[counts['counts'] > 12]['unique_values'].tolist()

In [54]:
Mod1E_results_final = pd.DataFrame(Mod1E_results_human[Mod1E_results_human['hit_symbol'].isin(high_counts)])

In [55]:
Mod1E_results_final['module'] = 'Mod1E'

In [61]:
Mod1E_results_final.head()

,hit_id,hit_symbol,input_id,input_symbol,score,module
13,HGNC:10290,RPA2,HGNC:1100,BRCA1,0,Mod1E
60,HGNC:1925,CHEK1,HGNC:1100,BRCA1,0,Mod1E
70,HGNC:3584,FANCC,HGNC:1100,BRCA1,0,Mod1E
92,HGNC:25764,RMI1,HGNC:1100,BRCA1,0,Mod1E
107,HGNC:10289,RPA1,HGNC:1101,BRCA2,0,Mod1E


In [57]:
all_results = pd.concat([Mod1A_results_human, Mod1B_results])

In [14]:
from Modules.StandardOutput import StandardOutput

In [58]:
so = StandardOutput(results=all_results.to_dict(orient='records'), input_object=input_object)

In [59]:
std_api_response_json = so.output_object

In [60]:
import requests

# get the URL for these results displayed in the RTX UI
RTX_UI_REQUEST_URL = "https://rtx.ncats.io/devED/api/rtx/v1/response/process"
to_post = {"options": ["Store", "ReturnResponseId"], "responses": [std_api_response_json]}
ui_url = requests.post(RTX_UI_REQUEST_URL, json=to_post)
print("Please visit the following website: https://rtx.ncats.io/?r=%s" % ui_url.json()['response_id'])

Please visit the following website: https://rtx.ncats.io/?r=567
